<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Region%20segmentation%20in%20synthetic%20lunar%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to segment a collection of nerves called the Brachial Plexus (BP) in ultrasound images



## 3. How to train a custom segmenter

# About the networks


1. UNet
  - https://arxiv.org/abs/1505.04597
  - https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
  - https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5

2. FPN
  - http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_Feature_Pyramid_Networks_CVPR_2017_paper.pdf
  - https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610
  - https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c


3. PSPNet
  - https://arxiv.org/abs/1612.01105
  - https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d
  - https://developers.arcgis.com/python/guide/how-pspnet-works/


4. Linknet
  - https://arxiv.org/pdf/1707.03718.pdf
  - https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/9_segmentation_models/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
#! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_cuda10.0.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'nerves': 1,
};
classes_to_train = ['background', 'nerves'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[580, 420])

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1elNb1ARvaSSPXRPIaQbuv28KhqH5ZkU6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1elNb1ARvaSSPXRPIaQbuv28KhqH5ZkU6" -O seg_nerve_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq seg_nerve_trained.zip

In [ ]:
gtf.Model_Params(model="Linknet", backbone="efficientnetb3", path_to_model='seg_nerve_trained/best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
import cv2
import numpy as np

def overlay_images(img_path):
    mask = np.load("output_mask.npy")[0].astype(np.uint8);
    
    img = cv2.imread(img_path, 1);
    img = cv2.resize(img, (608, 448))


    dst = cv2.addWeighted(img, 0.8, mask*255, 0.2, 0);
    cv2.imwrite("overlayed.jpg", dst);

    from IPython.display import Image
    Image(filename="overlayed.jpg") 

In [ ]:
img_name = "seg_nerve_trained/test/1.jpg";
gtf.Predict(img_name, vis=False);
overlay_images(img_name);
from IPython.display import Image
Image(filename="overlayed.jpg")

In [ ]:
img_name = "seg_nerve_trained/test/2.jpg";
gtf.Predict(img_name, vis=False);
overlay_images(img_name);
from IPython.display import Image
Image(filename="overlayed.jpg")

In [ ]:
img_name = "seg_nerve_trained/test/3.jpg";
gtf.Predict(img_name, vis=False);
overlay_images(img_name);
from IPython.display import Image
Image(filename="overlayed.jpg")

In [ ]:
img_name = "seg_nerve_trained/test/4.jpg";
gtf.Predict(img_name, vis=False);
overlay_images(img_name);
from IPython.display import Image
Image(filename="overlayed.jpg")

# Training your own segmenter 

## Dataset Credits
    - https://www.kaggle.com/c/ultrasound-nerve-segmentation/data

In [ ]:
! pip install kaggle

In [ ]:
! kaggle datasets download romainpessia/artificial-lunar-rocky-landscape-dataset

In [ ]:
! unzip -qq ultrasound-nerve-segmentation.zip -d nerve_dataset

In [ ]:
ls nerve_dataset/train

In [ ]:
import cv2
import numpy as np

In [ ]:
mask = cv2.imread("nerve_dataset/train/42_31_mask.tif", 1);
img = cv2.imread("nerve_dataset/train/42_31.tif", 1);
mask[np.where((mask==[255,255,255]).all(axis=2))] = [0,255,0]



dst = cv2.addWeighted(img, 0.8, mask, 0.2, 0);
cv2.imwrite("overlayed.jpg", dst);

from IPython.display import Image
Image(filename="overlayed.jpg") 

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread("nerve_dataset/train/42_24.tif", 1)
cv2.imwrite("tmp.jpg", img)

In [ ]:
from IPython.display import Image
Image(filename="tmp.jpg") 

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread("nerve_dataset/train/42_24_mask.tif", 0)
img.shape

In [ ]:
np.unique(img)

In [ ]:
import os

full_list = sorted(os.listdir("nerve_dataset/train/"))
img_list = [];
mask_list = [];

for i in range(len(full_list)):
    if("mask" in full_list[i]):
        mask_list.append(full_list[i]);
    else:
        img_list.append(full_list[i]);

In [ ]:
sorted(mask_list)

In [ ]:
sorted(img_list)

In [ ]:
import os 

os.mkdir("nerve_dataset/Train/");
os.mkdir("nerve_dataset/Train/img/");
os.mkdir("nerve_dataset/Train/mask");

os.mkdir("nerve_dataset/Val/");
os.mkdir("nerve_dataset/Val/img/");
os.mkdir("nerve_dataset/Val/mask");

In [ ]:
import cv2
import numpy as np

from tqdm.notebook import tqdm

for i in tqdm(range(len(img_list))):
    img_path = "nerve_dataset/train/" + img_list[i];
    img = cv2.imread(img_path, 1);
    
    cv2.imwrite("nerve_dataset/Train/img/" + img_list[i].split(".")[0] + ".jpg", img);
    
for i in tqdm(range(100)):
    img_path = "nerve_dataset/train/" + img_list[i];
    img = cv2.imread(img_path, 1);
    
    cv2.imwrite("nerve_dataset/Val/img/" + img_list[i].split(".")[0] + ".jpg", img);
    

In [ ]:
from tqdm.notebook import tqdm

for i in tqdm(range(len(mask_list))):
    img_path = "nerve_dataset/train/" + img_list[i].split(".")[0] + "_mask.tif";
    img = cv2.imread(img_path, 0);
    img[img > 0] = 1;
    
    cv2.imwrite("nerve_dataset/Train/mask/" + img_list[i].split(".")[0] + ".jpg", img);
    
for i in tqdm(range(100)):
    img_path = "nerve_dataset/train/" + img_list[i].split(".")[0] + "_mask.tif";
    img = cv2.imread(img_path, 0);
    img[img > 0] = 1;
    
    cv2.imwrite("nerve_dataset/Val/mask/" + img_list[i].split(".")[0] + ".jpg", img);
    

In [ ]:
from IPython.display import Image
Image(filename="nerve_dataset/train_img/42_24.jpg")

In [ ]:
img = cv2.imread("nerve_dataset/train_mask/42_24.jpg", 0);
cv2.imwrite("tmp_mask.jpg", img*255);
from IPython.display import Image
Image(filename="tmp_mask.jpg")

# Monk Format

## Dataset Directory Structure

      root_dir
          |
          | 
          |         
          |----train_img_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on) 
          |
          |----train_mask_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_img_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_mask_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)

          
          

# Sample Dataset Credits

    credits: https://www.kaggle.com/c/ultrasound-nerve-segmentation/data

In [ ]:
DATA_DIR = 'nerve_dataset/'

# Training

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from train_segmentation import Segmenter

In [ ]:
gtf = Segmenter();

In [ ]:
img_dir = "nerve_dataset/Train/img/";
mask_dir = "nerve_dataset/Train/mask/";

In [ ]:
classes_dict = {
    'background': 0, 
    'nerves': 1,
};
classes_to_train = ['background', 'nerves'];

In [ ]:
gtf.Train_Dataset(img_dir, mask_dir, classes_dict, classes_to_train)

In [ ]:
img_dir = "nerve_dataset/Val/img/";
mask_dir = "nerve_dataset/Val/mask/";

In [ ]:
gtf.Val_Dataset(img_dir, mask_dir)

In [ ]:
gtf.List_Backbones();

In [ ]:
gtf.Data_Params(batch_size=2, backbone="efficientnetb3", image_shape=[580, 420])

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model="Linknet")

In [ ]:
gtf.Train_Params(lr=0.001)

In [ ]:
gtf.Setup();

In [ ]:
gtf.Train(num_epochs=5);

In [ ]:
gtf.Visualize_Training_History();

# Inference

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'nerves': 1,
};
classes_to_train = ['nerves'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[580, 420])

In [ ]:
gtf.Model_Params(model="Linknet", backbone="efficientnetb3", path_to_model='best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
gtf.Predict("nerve_dataset/train_img/42_23.jpg", vis=True);

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread("nerve_dataset/train/42_23_mask.tif", 0)
cv2.imwrite("tmp.jpg", img)

from IPython.display import Image
Image(filename="tmp.jpg") 